In [2]:
import re
import mmcv
from sklearn.metrics import average_precision_score
import numpy as np
from typing import Tuple

def get_epoch(file_path: str):
# Read the file
  with open(file_path, "r") as file:
      content = file.read()

  # Use regex to extract the number after "mAP:"
  match = re.search(r"mAP:\s*(\d+)", content)
  if match:
      epoch = int(match.group(1))
      return epoch
  else:
      print("Epoch not found.")

def get_gt_pred(file_path: str):
    output_and_gt = mmcv.load(file_path)
    return np.array(output_and_gt[0]['outputs']), np.array(output_and_gt[0]['gt_labels'])
# import numpy as np
# outputs_result = None
# gt_labels_result = None
# for i in range(1,5):
#     file_path = f"coco/coco_uniform_bce_group{i}.txt"
#     epoch = get_epoch(file_path)
#     outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_group{i}/gt_and_results_e{epoch}.pkl")
#     if outputs_result is None:
#         outputs_result = outputs
#     else:
#         outputs_result = np.hstack((np.array(outputs_result), np.array(outputs)))
#     if gt_labels_result is None:
#         gt_labels_result = gt_labels
#     else:
#         gt_labels_result = np.hstack((np.array(gt_labels_result), np.array(gt_labels)))
    

def calculate_mAP(all_preds: np.ndarray, all_labels: np.ndarray) -> Tuple[np.float64, list]:
    label_cnt = len(all_labels[0])
    mAP = 0
    mAP_per_label = []
    # print(label_cnt)
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    for i in range(label_cnt):
        # print(np.sum(all_labels[:, i]) > 0)
        AP = average_precision_score(all_labels[:, i], all_preds[:, i])
        mAP_per_label.append(AP)
        mAP += AP

    mAP /= label_cnt
    APs = average_precision_score(all_labels, all_preds, average=None) 
    mAP = APs.mean()

    return mAP, mAP_per_label
def get_3_split_map(outputs_result, gt_labels_result):
    map_score1 =  calculate_mAP( np.array(outputs_result)[:, :22], np.array(gt_labels_result)[:, :22])
    # print(f"head mAP: {map_score1[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score2 =  calculate_mAP( np.array(outputs_result)[:, 22:55], np.array(gt_labels_result)[:, 22:55])
    # print(f"mid mAP: {map_score2[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score3 =  calculate_mAP( np.array(outputs_result)[:, 55:], np.array(gt_labels_result)[:, 55:])

    total = calculate_mAP( np.array(outputs_result), np.array(gt_labels_result))
    # print(f"tail mAP: {map_score3[0]:.4f}")
    print([round(map_score1[0], 4), round(map_score2[0], 4), round(map_score3[0], 4), round(total[0], 4)])
    # print(np.array(map_score[1]))


# get_3_split_map(outputs_result, gt_labels_result)

/home/mark/Desktop/工研院/DistributionBalancedLoss/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mark/Desktop/工研院/DistributionBalancedLoss/.venv/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [15]:
import ast
import pandas as pd
original_position = pd.read_csv("/home/mark/Desktop/工研院/multi-label_classification/data/coco/group2_original_positions_per_step.csv")
original_position['original_positions'] = original_position['original_positions'].apply(ast.literal_eval)
def get_original_index(step):
    if step in original_position['step'].values:
        return original_position.loc[original_position['step'] == step, 'original_positions'].values[0]
    else:
        raise ValueError(f"Step {step} not found.")

In [18]:
from tqdm import tqdm
best_score_list = []
range_left, range_right = 11, 50
for ri in tqdm(range(4, 11)):
    work_dir_str = f"LT_coco_resnet50_pfc_DB_uniform_bce_35_6_CVIR_group2_step{ri}"
    best_score = 0
    best_epoch = 0
    epoch_list = []
    for i in range(1,81):
        # epoch = get_epoch(file_path)
        outputs, gt_labels = get_gt_pred(f"work_dirs/{work_dir_str}/gt_and_results_e{i}.pkl")
        epoch_list.append((outputs, gt_labels))
        map_score = calculate_mAP(outputs[:, get_original_index(ri)], gt_labels[:,get_original_index(ri) ])
        if map_score[0] > best_score:
            best_score = map_score[0]
            best_epoch = i
    best_score_list.append((best_score, best_epoch))



100%|██████████| 7/7 [00:36<00:00,  5.19s/it]


In [19]:
best_score_list

[(0.6047756157989091, 37),
 (0.6038071354207358, 42),
 (0.6017351995485056, 53),
 (0.5995358795664304, 50),
 (0.5999955079244591, 53),
 (0.6008709706791012, 78),
 (0.6008362300256025, 56)]

In [18]:
best_score_list

[0.6229675280352791]

In [24]:
best_epoch

28

In [112]:
outputs[:,get_original_index(10)].shape

(5000, 39)

In [111]:
best_score_list

[(0.6047600344875798, 36),
 (0.6058464672269758, 32),
 (0.6017389176220953, 38),
 (0.6045177137242295, 36),
 (0.6045391143791078, 50),
 (0.6047300344764673, 42),
 (0.6023692674625177, 48),
 (0.6033794152990145, 53),
 (0.6009239422498157, 52),
 (0.6019962043015692, 63)]

In [57]:
best_score_list

[(0.6244537242756895, 9),
 (0.631269683207284, 10),
 (0.6272452772954641, 15),
 (0.6298287881205433, 13),
 (0.6319336448824721, 16),
 (0.6304857578304011, 15),
 (0.6267199275707046, 17),
 (0.6254485757250204, 19),
 (0.6228322864000027, 20),
 (0.6241215528569193, 21),
 (0.6213801213078801, 29),
 (0.6235290169708279, 22),
 (0.6232340825305664, 28),
 (0.6228016584622147, 26),
 (0.6218446863189525, 35),
 (0.6229577895391062, 32),
 (0.6228398396331677, 30)]

In [27]:
last = outputs[:,range_left: 11 ] 

In [28]:
(outputs[:, -30:] == last).every()

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [21]:
best_score_list

[(0.6302228720370208, 68),
 (0.6288683594595657, 71),
 (0.6227333914497694, 58),
 (0.6216182740062414, 64),
 (0.6267958467001103, 74),
 (0.6269370829145641, 72),
 (0.6111917834746652, 77),
 (0.6110707802614683, 78),
 (0.6156081032266659, 68),
 (0.5972553884034454, 75),
 (0.586243483137789, 73),
 (0.5809931827375383, 77)]

In [10]:
best_score_list

[(0.6244537242756895, 9),
 (0.631269683207284, 10),
 (0.6272452772954641, 15),
 (0.6298287881205433, 13),
 (0.6319336448824721, 16),
 (0.6304857578304011, 15),
 (0.6267199275707046, 17),
 (0.6254485757250204, 19),
 (0.6228322864000027, 20),
 (0.6241215528569193, 21),
 (0.6213801213078801, 29),
 (0.6235290169708279, 22),
 (0.6232340825305664, 28),
 (0.6228016584622147, 26),
 (0.6218446863189525, 35),
 (0.6229577895391062, 32),
 (0.6228398396331677, 30)]

## best group1 index 5 (0.6319336448824721, 16)

In [83]:
selected_indexes = [
    28,  # suitcase
    34,  # baseball glove
    78,  # teddy bear
    7,   # boat
    38,  # tennis racket
    14,  # bird
    6,   # train
    17,  # horse
    35,  # skateboard
    54,  # donut
    76,  # scissors
    51,  # hot dog
    12,  # stop sign
    30,  # skis
    10,  # fire hydrant
    29,  # frisbee
    79,  # toothbrush
    19,  # cow
    33,  # kite
    36,  # surfboard
    4,   # airplane
    67,  # toaster
    13,  # parking meter
    20,  # elephant
    31,  # snowboard
    18,  # sheep
    22,  # zebra
    23,  # giraffe
    21,  # bear
    77   # hair drier
]

# selected_indexes = [
#     0,   # person
#     57,  # dining table
#     56,  # chair
#     41,  # cup
#     39,  # bottle
#     44,  # bowl
#     70,  # book
#     2,   # car
#     68,  # sink
#     43,  # knife
#     60   # tv
# ]


In [13]:
best_epoch = 0
best_score = 0

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def calculate_mAP(all_preds: np.ndarray, all_labels: np.ndarray) -> Tuple[np.float64, list]:
    all_preds = sigmoid(all_preds)
    label_cnt = len(all_labels[0])
    mAP = 0
    mAP_per_label = []
    # print(label_cnt)
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    for i in range(label_cnt):
        # print(np.sum(all_labels[:, i]) > 0)
        AP = average_precision_score(all_labels[:, i], all_preds[:, i])
        mAP_per_label.append(AP)
        mAP += AP

    mAP /= label_cnt
    APs = average_precision_score(all_labels, all_preds, average=None) 
    mAP = APs.mean()

    return mAP, mAP_per_label
for i in range(16,17):
    # epoch = get_epoch(file_path)
    # outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group1_l0_r5/gt_and_results_e{i}.pkl")
    outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_classaware_bce/gt_and_results_e6.pkl")
    # epoch_list.append((outputs, gt_labels))
    map_score = calculate_mAP(outputs[:, :], gt_labels[:, :])
    if map_score[0] > best_score:
        best_score = map_score[0]
        best_epoch = i

print(best_score, best_epoch)

0.5967825588474797 16


In [16]:
outputs.min()

-9.805365

In [85]:
outputs[:,selected_indexes].shape

(5000, 30)

## score of group3 in bce all  0.5754014458097789 80

In [35]:
outputs[:,selected_indexes].shape

(5000, 30)

In [78]:
class_split = mmcv.load("appendix/coco/longtail2017/class_split.pkl")

outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_classaware_DBloss/gt_and_results_e8.pkl")

head_score = calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])[0]
mid_score = calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])[0]
tail_score = calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])[0]


# head_score = calculate_mAP(outputs[:, :22], gt_labels[:, :22])[0]
# mid_score = calculate_mAP(outputs[:, 22:55], gt_labels[:, 22:55])[0]
# tail_score = calculate_mAP(outputs[:, 55:], gt_labels[:, 55:])[0]
total_score = calculate_mAP(outputs, gt_labels)[0]

print(f"head mAP: {head_score:.4f}")
print(f"mid mAP: {mid_score:.4f}")
print(f"tail mAP: {tail_score:.4f}")
print(f"total mAP: {total_score:.4f}")


head mAP: 0.5328
mid mAP: 0.6091
tail mAP: 0.5618
total mAP: 0.5733


In [79]:
gt_labels.sum(axis=0)

array([2693,  149,  535,  159,   97,  189,  157,  250,  121,  191,   86,
         69,   37,  235,  125,  184,  177,  128,   65,   87,   89,   49,
         85,  101,  228,  174,  292,  145,  105,   84,  120,   49,  169,
         91,   97,  100,  127,  149,  167,  379,  110,  390,  155,  181,
        153,  314,  103,   76,   98,   85,   71,   81,   51,  153,   62,
        124,  580,  195,  172,  149,  501,  149,  207,  183,   88,  145,
        106,  214,   54,  115,    8,  187,  101,  230,  204,  137,   28,
         94,    9,   34])

In [81]:
import pandas as pd
df = pd.read_csv("/home/mark/Desktop/工研院/multi-label_classification/data/coco/test_data.csv", index_col=0)
np.array(df.sum(axis=0))

array([2693,  149,  535,  159,   97,  189,  157,  250,  121,  191,   86,
         69,   37,  235,  125,  184,  177,  128,   65,   87,   89,   49,
         85,  101,  228,  174,  292,  145,  105,   84,  120,   49,  169,
         91,   97,  100,  127,  149,  167,  379,  110,  390,  155,  181,
        153,  314,  103,   76,   98,   85,   71,   81,   51,  153,   62,
        124,  580,  195,  172,  149,  501,  149,  207,  183,   88,  145,
        106,  214,   54,  115,    8,  187,  101,  230,  204,  137,   28,
         94,    9,   34])

In [102]:
df.columns

Index(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
       'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
       'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
       'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
       'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
       'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
       'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
       'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
       'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
       'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
       'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
       'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
       'scissors', 'teddy bear', 'hair drier', 'toothbrush'],
      dtype='object')

In [105]:
# Full COCO class list
coco_classes = "person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,fire hydrant,stop sign,parking meter,bench,bird,cat,dog,horse,sheep,cow,elephant,bear,zebra,giraffe,backpack,umbrella,handbag,tie,suitcase,frisbee,skis,snowboard,sports ball,kite,baseball bat,baseball glove,skateboard,surfboard,tennis racket,bottle,wine glass,cup,fork,knife,spoon,bowl,banana,apple,sandwich,orange,broccoli,carrot,hot dog,pizza,donut,cake,chair,couch,potted plant,bed,dining table,toilet,tv,laptop,mouse,remote,keyboard,cell phone,microwave,oven,toaster,sink,refrigerator,book,clock,vase,scissors,teddy bear,hair drier,toothbrush".split(",")

# Target list to find indexes for
target_items = [
    "person", "dining table", "chair", "cup", "bottle",
    "bowl", "book", "car", "sink", "knife", "tv"
]

# Normalize function: lowercase and replace underscores with spaces
def normalize(label):
    return label.strip().lower().replace("_", " ")

# Build normalized list
normalized_coco = [normalize(label) for label in coco_classes]
normalized_targets = [normalize(label) for label in target_items]

# Get indexes
indexes = [normalized_coco.index(label) for label in normalized_targets]

# Output result
print("Target indexes:", indexes)


Target indexes: [0, 60, 56, 41, 39, 45, 73, 2, 71, 43, 62]


In [106]:
np.array(df.columns)[[0, 60, 56, 41, 39, 45, 73, 2, 71, 43, 62]]

array(['person', 'dining table', 'chair', 'cup', 'bottle', 'bowl', 'book',
       'car', 'sink', 'knife', 'tv'], dtype=object)

In [ ]:
"all": [0.5704, 0.636, 0.5447, 0.5894],

In [55]:
calculate_mAP(outputs[:, :11], gt_labels[:, :11])[0]

0.6065970589127846

In [5]:
outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_all/gt_and_results_e{1}.pkl")

In [7]:
gt_labels

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [63]:
gt_labels.sum(axis=0)

array([2693,  149,  535,  159,   97,  189,  157,  250,  121,  191,   86,
         69,   37,  235,  125,  184,  177,  128,   65,   87,   89,   49,
         85,  101,  228,  174,  292,  145,  105,   84,  120,   49,  169,
         91,   97,  100,  127,  149,  167,  379,  110,  390,  155,  181,
        153,  314,  103,   76,   98,   85,   71,   81,   51,  153,   62,
        124,  580,  195,  172,  149,  501,  149,  207,  183,   88,  145,
        106,  214,   54,  115,    8,  187,  101,  230,  204,  137,   28,
         94,    9,   34])

In [74]:
result = mmcv.load("work_dirs/LT_coco_resnet50_pfc_DB/gt_and_results_e10.pkl")
np.array(result[0]['gt_labels']).sum(axis=0)

array([2693,  149,  535,  159,   97,  189,  157,  250,  121,  191,   86,
         69,   37,  235,  125,  184,  177,  128,   65,   87,   89,   49,
         85,  101,  228,  174,  292,  145,  105,   84,  120,   49,  169,
         91,   97,  100,  127,  149,  167,  379,  110,  390,  155,  181,
        153,  314,  103,   76,   98,   85,   71,   81,   51,  153,   62,
        124,  580,  195,  172,  149,  501,  149,  207,  183,   88,  145,
        106,  214,   54,  115,    8,  187,  101,  230,  204,  137,   28,
         94,    9,   34])

In [6]:
# head_score = calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])[0]
# mid_score = calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])[0]
# tail_score = calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])[0]

outputs1, gt_labels1 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group1_l0_r5/gt_and_results_e16.pkl")
outputs2, gt_labels2 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group2/gt_and_results_e30.pkl")
outputs3, gt_labels3 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group3/gt_and_results_e58.pkl")


# outputs1, gt_labels1 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group1_v1/gt_and_results_e5.pkl")
# outputs2, gt_labels2 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group2_v1/gt_and_results_e14.pkl")
# outputs3, gt_labels3 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group3_v1/gt_and_results_e43.pkl")


outputs = np.hstack((outputs1[:, :11], outputs2, outputs3))
gt_labels = np.hstack((gt_labels1[:, :11], gt_labels2, gt_labels3))

head_score = calculate_mAP(outputs[:, :22], gt_labels[:, :22])[0]
mid_score = calculate_mAP(outputs[:, 22:55], gt_labels[:, 22:55])[0]
tail_score = calculate_mAP(outputs[:, 55:], gt_labels[:, 55:])[0]
total_score = calculate_mAP(outputs, gt_labels)[0]

print(f"head mAP: {head_score:.4f}")
print(f"mid mAP: {mid_score:.4f}")
print(f"tail mAP: {tail_score:.4f}")
print(f"total mAP: {total_score:.4f}")
print([round(head_score, 4), round(mid_score, 4), round(tail_score, 4), round(total_score, 4)])


head mAP: 0.5772
mid mAP: 0.6504
tail mAP: 0.6218
total mAP: 0.6213
[0.5772, 0.6504, 0.6218, 0.6213]


In [3]:
# head_score = calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])[0]
# mid_score = calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])[0]
# tail_score = calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])[0]

# outputs1, gt_labels1 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group1_l0_r5/gt_and_results_e16.pkl")
# outputs2, gt_labels2 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group2/gt_and_results_e30.pkl")
# outputs3, gt_labels3 = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group3/gt_and_results_e58.pkl")


outputs1, gt_labels1 = get_gt_pred(f"work_dirs/LT_voc_resnet101_pfc_DB_uniform_bce_35_6_group1/gt_and_results_e7.pkl")
outputs2, gt_labels2 = get_gt_pred(f"work_dirs/LT_voc_resnet101_pfc_DB_uniform_bce_35_6_group2/gt_and_results_e13.pkl")
outputs3, gt_labels3 = get_gt_pred(f"work_dirs/LT_voc_resnet101_pfc_DB_uniform_bce_35_6_group3/gt_and_results_e60.pkl")


outputs = np.hstack((outputs1[:, :], outputs2, outputs3))
gt_labels = np.hstack((gt_labels1[:, :], gt_labels2, gt_labels3))

head_score = calculate_mAP(outputs[:, :6], gt_labels[:, :6])[0]
mid_score = calculate_mAP(outputs[:, 6:12], gt_labels[:, 6:12])[0]
tail_score = calculate_mAP(outputs[:, 12:], gt_labels[:, 12:])[0]
total_score = calculate_mAP(outputs, gt_labels)[0]

print(f"head mAP: {head_score:.4f}")
print(f"mid mAP: {mid_score:.4f}")
print(f"tail mAP: {tail_score:.4f}")
print(f"total mAP: {total_score:.4f}")
print([round(head_score, 4), round(mid_score, 4), round(tail_score, 4), round(total_score, 4)])


head mAP: 0.7502
mid mAP: 0.8636
tail mAP: 0.8939
total mAP: 0.8417
[0.7502, 0.8636, 0.8939, 0.8417]


In [119]:
outputs.shape

(5000, 80)

In [9]:
class_split = mmcv.load("appendix/coco/longtail2017/class_split.pkl")

from tqdm import tqdm
best_head_e = 0
best_head_score = 0
best_mid_e = 0
best_mid_score = 0
best_tail_e = 0
best_tail_score = 0
best_total_e = 0
best_total_score = 0

for i in tqdm(range( 1, 81)):
    outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_35_6_group1_l0_r18/gt_and_results_e{i}.pkl")

    # head_score = calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])[0]
    # mid_score = calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])[0]
    # tail_score = calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])[0]


    head_score = calculate_mAP(outputs[:, :22], gt_labels[:, :22])[0]
    mid_score = calculate_mAP(outputs[:, 11:50], gt_labels[:, 11:50])[0]
    tail_score = calculate_mAP(outputs[:, 55:], gt_labels[:, 55:])[0]
    total_score = calculate_mAP(outputs, gt_labels)[0]
    if tail_score > best_tail_score:
        best_tail_score = tail_score
        best_tail_e = i
    if head_score > best_head_score:
        best_head_score = head_score
        best_head_e = i
    if mid_score > best_mid_score:
        best_mid_score = mid_score
        best_mid_e = i
    if total_score > best_total_score:
        best_total_score = total_score
        best_total_e = i
    # print(f"head mAP: {head_score:.4f}")
    # print(f"mid mAP: {mid_score:.4f}")
    # print(f"tail mAP: {tail_score:.4f}")
    # print(f"total mAP: {total_score:.4f}")
    # print([head_score])
    print(f"{100*total_score:.2f} & {100*head_score:.2f} & {100*mid_score:.2f} & {100*tail_score:.2f}")

print(f"Best head mAP: {best_head_score:.4f} at epoch {best_head_e}")
print(f"Best mid mAP: {best_mid_score:.4f} at epoch {best_mid_e}")
print(f"Best tail mAP: {best_tail_score:.4f} at epoch {best_tail_e}")
print(f"Best total mAP: {best_total_score:.4f} at epoch {best_total_e}")

# print(f"{best_total}")

  1%|▏         | 1/80 [00:00<00:32,  2.43it/s]

6.37 & 14.60 & 5.09 & 2.28


  2%|▎         | 2/80 [00:00<00:32,  2.40it/s]

9.92 & 24.48 & 8.33 & 2.40


  4%|▍         | 3/80 [00:01<00:28,  2.73it/s]

...

58.33 & 56.57 & 59.96 & 53.01


 99%|█████████▉| 79/80 [00:24<00:00,  4.18it/s]

58.25 & 56.42 & 59.79 & 53.00


100%|██████████| 80/80 [00:24<00:00,  3.28it/s]

58.32 & 56.52 & 59.85 & 53.12
Best head mAP: 0.5736 at epoch 44
Best mid mAP: 0.5996 at epoch 78
Best tail mAP: 0.5319 at epoch 73
Best total mAP: 0.5833 at epoch 78
